# Simulating a Radiology Clinic

A hospital’s radiology department is suffering from poor workflow efficiency. You are brought in as a
consultant to fix the problem. After several interviews with the radiology department personnel and
collecting a lot of relevant data, you develop a good understanding of how the department
workflow currently works: 



1.  Appointments are scheduled such that one patient is scheduled to arrive
every 30 minutes. However, in reality the patients arrive uniformly in the interval starting from 10
minutes before the scheduled arrival time to 10 minutes past the scheduled arrival time.
2.   Each
patient first fills up the consent forms. This usually takes only 2 minutes. However, if a patient gets
stuck somewhere in that process and has some questions, then the process of filling up the forms
consumes approximately 10 minutes. The patient does get stuck once in every 5 cases or so (so,
probability of getting stuck = 0.2).
3.   As soon as the forms are filled, the second step is to prepare the
patient for the radiology exam. This involves transporting the patient to the exam room, having
them change into the exam clothes, and administering any required medicines. The time taken for
preparing the patient for the exam has an exponential distribution with expected value of 15
minutes.
4.   While the patient is being prepared for the radiology exam, the exam room is also
prepared simultaneously. Preparing the exam room takes exactly 10 minutes.
5.   Once the patient and
the room both are ready, the exam is conducted. The exam lasts for either 8, 10 or 12 minutes with
equal probability.
6.   As soon as an exam is finished, we can start preparing the room for the next exam. As soon as an exam is finished, we can also start preparing the next patient if the patient has finished the paperwork.

In this homework you will build a discrete-event simulation model of the radiology department workflow. Let's start by listing out the variables we need. We will be generating some number of patients in this simulation - let's call this `n_max` - and we'll be keeping track of their progress through the radiology department and recording metrics of interest. Each of our metrics will be a list of length `n_max`, with each element of the list initialized to zero - we can do this by writing `[0]*n_max`, just as in the Coffee Shop and Repair Facility simulations. Here are the metrics we will want to keep track of:

```
  #arrival time of patient n 
  PA=[0]*n_max
  #time taken for patient n to fill out forms
  FF=[0]*n_max
  #time taken for patient n to get prepared for exam
  PP=[0]*n_max
  #time taken to prepare exam room for patient n
  EP=[0]*n_max
  #time taken for exam for patient n
  E=[0]*n_max
  #start time of exam for patient n
  ES=[0]*n_max
  #end time of exam for patient n
  EE=[0]*n_max
  #time when the exam room gets ready for patient n
  ER=[0]*n_max 
  #time when patient n gets ready for the exam
  PR=[0]*n_max
  #time when the patient starts getting prepared for the exam 
  PPS=[0]*n_max
```


Now, lets start by building up the simulation piece by piece. First, we need a function to generate an arrival time. Suppose that one patient arrives every 30 minutes as scheduled. If patients are numbered `0`, `1`, `2`, and so on, then we could find the arrival time of patient `n` as `30*n` (so that, for example, patient `0` arrives at time `0`. However, note that there is some random component to a patients arrival. They arrive uniformly in the interval from ten minutes before to ten minutes after their scheduled time. 

How can we model this? If we can generate a random number uniformly from 0 to 1 by calling `random.random()`, as we saw in lecture, we just need to ensure that the lower bound of arrival time corresponds to the case when `random.random()` returns 0, and the upper bound of arrival time corresponds to the case when `random.random()` returns 1.

The cell below defines a function such patients arrival uniformly in an interval of 15 minutes before to 15 minutes after their scheduled time. If `random.random()` returns 0, we have `30*n -15 + 0`, or `-15` for patient `0`, and if `random.random()` returns 1, we have `30*n -15 + 30`, or `15` for patient `0`. 

**Change the code below so that patients arrival at the correct interval of 10 minutes before to 10 minutes after their start time**

In [79]:
import random

def get_arrival_time(n):
  # arrival_time=30*n-15+30*random.random()
  arrival_time=30*n-10+20*random.random()
  return arrival_time

Now, **call the function you've created a few times for patient `0` and view the output by running the cell below**. Are the values falling between -10 and 10? 

In [80]:
get_arrival_time(0)

-5.41247585322326

Next, we need one to generate the time needed for filling out forms. We know that patients can fill out the forms in 2 minutes 80% of the time. But 1 out of every 5 times, they get stuck, and the process takes 10 minutes. 

How do we model this? If we can generate a random number uniformly from 0 to 1 using `random.random()`, we just need to make sure we set `fill_forms_time` to 2 for 80% of the possible values between 0 and 1, and to 10 for the 20% of possible values. We can do this with an `if` statement - probably the easiest option is to set `fill_forms_time` equal to 2 if the random number is less than some value, and equal to 10 otherwise. 

The cell below defines a function such that 40% of the time patients take 2 minutes to fill out the forms, and 60% of the time they take 10 minutes to fill out the forms. **Change the function such that they fill out the forms in 2 minutes 80% of the time, and get stuck 20% of the time.**

In [81]:
def get_fill_forms_time():
   r=random.random()
   # if(r<0.4): 
   if(r<0.8): 
     fill_forms_time=2
   else:
     fill_forms_time=10
   return fill_forms_time

Try calling the function a few times below:

In [82]:
get_fill_forms_time()

10

We also need a function that generates the time needed for patient preparation. We know that this time follows an exponential distribution with a mean of 15 minutes. Recall from lecture that we can generate these variables as $-λ \log(random.random())$, where lambda is the distribution mean. The cell below generates exponentially distributed wait times with a mean of 10 minutes. **Change the code so that it has a mean wait time of 15 minutes**.

In [83]:
import math

def get_exam_patient_prep_time():
  # patient_prep_time = -10*math.log(random.random())
  patient_prep_time = -15*math.log(random.random())
  return patient_prep_time

Try generating some of these exponentially ditributed times by running the cell below to call the function. 

In [84]:
get_exam_patient_prep_time()

10.53266383898222

Exam room prep time is just a constant: 10 minutes. **Run the cell below to define a simple function that always returns 10**. This is a very simple function, but defining at as a function allows us to easily change the simulation later.  

In [85]:
def get_exam_room_prep_time():
   return 10

Next, we need to define exam time. Exams last 8 minutes, 10 minutes or 12 minutes with equal probability (in other words, each of those options occurs 1/3 of the time). This is quite similar to the form filling time, except that now we have 3 possible outcomes instead of 2, so we will have to use an `elif` statement, such that if the random number is greater than a certain threshold, but less than another, the outcome will be 10 minutes. The code below generates wait times that are 8 minutes 1/4 of the time, 10 minutes 1/4 of the time, and 12 minutes 50% of the time. **Adjust this code so that each of the outcomes occurs with equal probability**.

In [86]:
def get_exam_time():
   #calculate exam time 
   r=random.random(); 
   # if (r<(1/4)):
   if (r<(1/3)):
    exam_time=8 
   # elif (r<1/2): 
   elif (r<2/3): 
    exam_time=10 
   else: 
    exam_time=12 
   return exam_time 

Finally, lets make a function for patient prep start time. We know that patients can start being prepped in the exam. We know that this can occur when the patient is done with his forms AND the exam room is free. In other words, this happens at whatever time is later for these two events. We can determine this using the `max` function. We define our function for a given patient `n`, and pass it the vector `PA` (patient arrival times) and  `FF` (patient form filling times. The time the patient is ready to begin preparation will be `PA[n]+FF[n]`. We need to also know when the exam room is free, which we can get by looking at the exam end time of the previous patient, `EE[n-1]`. We can then take the maximum of these two values to get the preparation start time.  **Run the cell below to define this function.**

In [87]:
def get_patient_prep_start_time(PA,FF,EE,n):
   return max(PA[n]+FF[n],EE[n-1])

We are now ready to put these function together in a full simulation. **Read through the code below to see where each of these functions are called, where their output is stored, and follow the path of computation from patient arrival time `PA` to exam end time `EE`. Then run this cell to define this function.**

In [88]:
import math
import random


def run_radiology_sim(n_max=1000):

##variables to keep track of  
  #arrival time of patient n 
  PA=[0]*n_max
  #time taken for patient n to fill out forms
  FF=[0]*n_max
  #time taken for patient n to get prepared for exam
  PP=[0]*n_max
  #time taken to prepare exam room for patient n
  EP=[0]*n_max
  #time taken for exam for patient n
  E=[0]*n_max
  #start time of exam for patient n
  ES=[0]*n_max
  #end time of exam for patient n
  EE=[0]*n_max
  #time when the exam room gets ready for patient n
  ER=[0]*n_max 
  #time when patient n gets ready for the exam
  PR=[0]*n_max
  #time when the patient starts getting prepared for the exam 
  PPS=[0]*n_max

##run simulation
  for n in range(0,n_max):
    #calculate arrival time
    PA[n]=get_arrival_time(n)

    #calculate time to fill out forms
    FF[n] = get_fill_forms_time()
    #calculate time taken to prep patient for exam
    PP[n]=get_exam_patient_prep_time()
    #record time to prepare exam room 
    EP[n]=get_exam_room_prep_time()
    #calculate exam time 
    E[n] = get_exam_time()

    # calculate when exam is ready (ER) and patient starts getting prepared (PPS)    
    if (n==0): #for first patient...
      #exam room ready once it is prepared (EP)
      ER[n]=EP[n] 
      #patient starts getting prepped at the time they arrive plus the time to fill forms
      PPS[n]=PA[n]+FF[n] 
    else: #for all later patients...
      #exam room is ready at the time previous patient exits (EE) plus exam room prep time (EP)
      ER[n]=EE[n-1]+EP[n] 
      #patient starts getting prepped at whichever comes later, the time they arrive plus the time to fill forms, 
      #or, the previous patient has left the exam room (EE[n-1])
      PPS[n]=get_patient_prep_start_time(PA,FF,EE,n)

    #time when patient is ready for exam is time they start getting prepped (PPS) plus prep time (PP)
    PR[n]=PPS[n]+PP[n] 
    #start time of the exam is whichever comes later, when the exam room is ready (ER) and the patient is ready (PR)
    ES[n]=max(ER[n],PR[n])
    #end of exam time is start of exam time (ES) plus exam prep time (EE)
    EE[n]=ES[n]+E[n]
  return EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E


Now, let's test the simulation 1000 patients in steady state (we will assume, for simplicity, that the radiology department works
24/7). We will ignore the first 100 patients as warm-up period.

We want to calculate five different aggregate metrics: 

I. Average time spent by each patient in the radiology department. 

II. Average time from a patient’s scheduled arrival time to patient’s actual exit
from the radiology department. 

III. Percentage of time for which the exam room is being used for the exam
itself.

IV. Percentage of time for which the exam room is used for preparing the
patient.

V. Percentage of time for which the exam room is used for preparing the exam
room.

The code below runs the simulation function you defined above `M` times, each time recording each of these metrics. It then averages each of these metrics across all runs of the simulation. **Run the cell below and examine the averages.** How does the average time spent by each patient compare to the scheduled time of 30 minutees? How does the average time from a patient's scheduled arrival time to a patient's exit compare to the scheduled time of 30 minutes? 

In [89]:
from numpy import mean, array

#define a random seed (so that we get the same outcome every time. )
random.seed(42)

#total number of simulations to run
M=10000
#number of patients per simulation
n_max = 1000

#average time per patient in the radiology department, for each simulation run
avg_time=[0]*M
#average time from scheduled appointment start to patient exit, for each simulation run
avg_time_sch=[0]*M
#percentage of time exam room is being used for the exam, for each simulation run
perc_exam_time=[0]*M 
#percentage of time the exam room is used for preparing the patient
perc_ptprep_time=[0]*M
#percentatage of time the exam room is used for preparing the exam room
perc_roomprep_time=[0]*M

#patient scheduled arrival times (every 30 minutes), for n_max patients
PA_sch=list(range(0,(n_max)*30,30))

#patients to discard from calculations as warmup 
warmup = 100  

for m in range(0,M):
  EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E = run_radiology_sim(n_max=n_max)
  #to get average time, subtract all patient exam end times from patient arrival times, then average
  avg_time[m]=mean(array(EE[warmup:])-array(PA[warmup:]))
  #similarly, subtract all patient exam end times from patient scheduled arrival times, then average
  avg_time_sch[m]=mean(array(EE[warmup:])- array(PA_sch[warmup:]))
  #divide sum of all exam times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_exam_time=sum(E[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all patient prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_ptprep_time=sum(PP[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all exam prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_roomprep_time=sum(EP[warmup:])/(EE[-1]- PA[warmup])
  
  # if m is evenly divisible by 1000, print progress
  if m % 1000 == 0:
    print("run %s complete" % m)

##compute averages of metrics across all simulation runs
mean_avg_time = mean(avg_time) #aggregate metric I
mean_avg_time_sch = mean(avg_time_sch) #aggregate metric II
mean_perc_exam_time =mean(perc_exam_time) #aggregate metric III
mean_perc_ptprep_time = mean(perc_ptprep_time) #aggregate metric IV
mean_perc_roomprep_time =  mean(perc_roomprep_time) #aggregate metric V

print("Average time in dept: %s minutes" % mean_avg_time)
print("Average scheduled arrival to end exam: %s minutes" % mean_avg_time_sch)
print("Average percent exam time: %s" % (mean_perc_exam_time*100))
print("Average percent patient prep: %s" % (mean_perc_ptprep_time*100))
print("Average percent room prep: %s" % (mean_perc_roomprep_time*100))

run 0 complete
run 1000 complete
run 2000 complete
run 3000 complete
run 4000 complete
run 5000 complete
run 6000 complete
run 7000 complete
run 8000 complete
run 9000 complete
Average time in dept: 67.31991392646835 minutes
Average scheduled arrival to end exam: 67.31937874055117 minutes
Average percent exam time: 33.136841991329305
Average percent patient prep: 50.59767164804268
Average percent room prep: 33.336863170351414


In your role as a consultant, suppose you are evaluating, as one possible recommendation, the idea of having a nurse call the
patients ahead of their appointment to ensure that they arrive on time. It will cost you $10 per
patient. How much will this reduce the average time spent by the patients in the radiology
department? How much is the saving per dollar spent?

In order examine this do this, let's redefine our arrival time function. **Change the function definition below to remove the random element.** 

In [90]:
def get_arrival_time(n):
  arrival_time=30*n # -15+30*random.random()
  return arrival_time

Now, **rerun our simulations by running the cell below.** The code below is the same as two cells above, except that we have appended `_ontime` to all of the aggregate variable names, so that we can compare the numbers.    

In [91]:
from numpy import mean, array

#define a random seed (so that we get the same outcome every time. )
random.seed(42)

#total number of simulations to run
M=10000
#number of patients per simulation
n_max = 1000

#average time per patient in the radiology department, for each simulation run
avg_time=[0]*M
#average time from scheduled appointment start to patient exit, for each simulation run
avg_time_sch=[0]*M
#percentage of time exam room is being used for the exam, for each simulation run
perc_exam_time=[0]*M 
#percentage of time the exam room is used for preparing the patient
perc_ptprep_time=[0]*M
#percentatage of time the exam room is used for preparing the exam room
perc_roomprep_time=[0]*M

#patient scheduled arrival times (every 30 minutes), for n_max patients
PA_sch=list(range(0,(n_max)*30,30))

#patients to discard from calculations as warmup 
warmup = 100  

for m in range(0,M):
  EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E = run_radiology_sim(n_max=n_max)
  #to get average time, subtract all patient exam end times from patient arrival times, then average
  avg_time[m]=mean(array(EE[warmup:])-array(PA[warmup:]))
  #similarly, subtract all patient exam end times from patient scheduled arrival times, then average
  avg_time_sch[m]=mean(array(EE[warmup:])- array(PA_sch[warmup:]))
  #divide sum of all exam times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_exam_time=sum(E[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all patient prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_ptprep_time=sum(PP[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all exam prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_roomprep_time=sum(EP[warmup:])/(EE[-1]- PA[warmup])
  
  # if m is evenly divisible by 1000, print progress
  if m % 1000 == 0:
    print("run %s complete" % m)

##compute averages of metrics across all simulation runs
mean_avg_time_ontime = mean(avg_time) #aggregate metric I
mean_avg_time_sch_ontime = mean(avg_time_sch) #aggregate metric II
mean_perc_exam_time_ontime =mean(perc_exam_time) #aggregate metric III
mean_perc_ptprep_time_ontime = mean(perc_ptprep_time) #aggregate metric IV
mean_perc_roomprep_time_ontime =  mean(perc_roomprep_time) #aggregate metric V

print("Average time in dept: %s minutes" % mean_avg_time_ontime)
print("Average scheduled arrival to end exam: %s minutes" % mean_avg_time_sch_ontime)
print("Average percent exam time: %s" % (mean_perc_exam_time_ontime*100))
print("Average percent patient prep: %s" % (mean_perc_ptprep_time_ontime*100))
print("Average percent room prep: %s" % (mean_perc_roomprep_time_ontime*100))

run 0 complete
run 1000 complete
run 2000 complete
run 3000 complete
run 4000 complete
run 5000 complete
run 6000 complete
run 7000 complete
run 8000 complete
run 9000 complete
Average time in dept: 65.46308021127521 minutes
Average scheduled arrival to end exam: 65.46308021127521 minutes
Average percent exam time: 33.25163365123541
Average percent patient prep: 48.73224039861079
Average percent room prep: 33.229480664125994


How does this compare to our previous run? The cell below computes first the difference between averages, then the difference in averages divided by dollars invested. 

In [92]:
#difference in averages
print(mean_avg_time-mean_avg_time_ontime) 
#savings in seconds per dollar invested
print((mean_avg_time-mean_avg_time_ontime)*60/10) 

1.8568337151931331
11.141002291158799


Another possible recommendation is to have a nurse help the patients while filling the forms
so that they finish in exactly 2 minutes each time. This will cost $5 per patient. How much will this
reduce the average time spent by the patients in the radiology department? How much is the
saving per dollar spent?

First, we need to restore the original arrival time function. **Run the cell below to revert back to the original**. 

In [93]:
def get_arrival_time(n):
  arrival_time=30*n-10+20*random.random()
  return arrival_time

Now, let's adjust our forms function. **Change the function below so that patients finish the forms in exactly 2 minutes.** 

In [94]:
def get_fill_forms_time():
  # r=random.random()
  # if(r<0.4): 
  fill_forms_time=2
  # else:
  # fill_forms_time=10
  return fill_forms_time

We can now rerun our simulations. We now append `_easyforms` to the aggregate variables. How have the results changed? 

In [95]:
from numpy import mean, array

#define a random seed (so that we get the same outcome every time. )
random.seed(42)

#total number of simulations to run
M=10000
#number of patients per simulation
n_max = 1000

#average time per patient in the radiology department, for each simulation run
avg_time=[0]*M
#average time from scheduled appointment start to patient exit, for each simulation run
avg_time_sch=[0]*M
#percentage of time exam room is being used for the exam, for each simulation run
perc_exam_time=[0]*M 
#percentage of time the exam room is used for preparing the patient
perc_ptprep_time=[0]*M
#percentatage of time the exam room is used for preparing the exam room
perc_roomprep_time=[0]*M

#patient scheduled arrival times (every 30 minutes), for n_max patients
PA_sch=list(range(0,(n_max)*30,30))

#patients to discard from calculations as warmup 
warmup = 100  

for m in range(0,M):
  EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E = run_radiology_sim(n_max=n_max)
  #to get average time, subtract all patient exam end times from patient arrival times, then average
  avg_time[m]=mean(array(EE[warmup:])-array(PA[warmup:]))
  #similarly, subtract all patient exam end times from patient scheduled arrival times, then average
  avg_time_sch[m]=mean(array(EE[warmup:])- array(PA_sch[warmup:]))
  #divide sum of all exam times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_exam_time=sum(E[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all patient prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_ptprep_time=sum(PP[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all exam prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_roomprep_time=sum(EP[warmup:])/(EE[-1]- PA[warmup])
  
  # if m is evenly divisible by 1000, print progress
  if m % 1000 == 0:
    print("run %s complete" % m)

##compute averages of metrics across all simulation runs
mean_avg_time_easyforms = mean(avg_time) #aggregate metric I
mean_avg_time_sch_easyforms = mean(avg_time_sch) #aggregate metric II
mean_perc_exam_time_easyforms =mean(perc_exam_time) #aggregate metric III
mean_perc_ptprep_time_easyforms = mean(perc_ptprep_time) #aggregate metric IV
mean_perc_roomprep_time_easyforms =  mean(perc_roomprep_time) #aggregate metric V

print("Average time in dept: %s minutes" % mean_avg_time_easyforms)
print("Average scheduled arrival to end exam: %s minutes" % mean_avg_time_sch_easyforms)
print("Average percent exam time: %s" % (mean_perc_exam_time_easyforms*100))
print("Average percent patient prep: %s" % (mean_perc_ptprep_time_easyforms*100))
print("Average percent room prep: %s" % (mean_perc_roomprep_time_easyforms*100))

run 0 complete
run 1000 complete
run 2000 complete
run 3000 complete
run 4000 complete
run 5000 complete
run 6000 complete
run 7000 complete
run 8000 complete
run 9000 complete
Average time in dept: 64.93272188147648 minutes
Average scheduled arrival to end exam: 64.92996253751747 minutes
Average percent exam time: 33.24887802580771
Average percent patient prep: 48.7282018661826
Average percent room prep: 33.22672687455801


Now, **fill in the code below, using the code after the first simulation as a template, to compute the difference in averages and the savings in seconds per dollar invested.** 

In [96]:
#difference in averages
print(mean_avg_time-mean_avg_time_easyforms) 
#savings in seconds per dollar invested
print((mean_avg_time-mean_avg_time_easyforms)*60/10) 

2.3871920449918633
14.32315226995118


Another possible recommendation is to add an extra technician to prepare the exam room.
This will cost an extra $20 per patient but will cut the exam preparation time from 10 minutes down
to 5 minutes. How much will this reduce the average time spent by the patients in the radiology
department? How much is the saving per dollar spent?

First, restore the filling forms function by running the cell below. 

In [97]:
def get_fill_forms_time():
   r=random.random()
   if(r<0.8): 
     fill_forms_time=2
   else:
     fill_forms_time=10
   return fill_forms_time

Next, **modify the exam room prep time function below to cut the exam preparation time down to 5 minutes.** 

In [98]:
def get_exam_room_prep_time():
  # return 10
  return 5

Now, run the simulations again, by running the cell below, which simply adds `_fastexam` to all aggregate metric variable names. 

In [99]:
from numpy import mean, array

#define a random seed (so that we get the same outcome every time. )
random.seed(42)

#total number of simulations to run
M=10000
#number of patients per simulation
n_max = 1000

#average time per patient in the radiology department, for each simulation run
avg_time=[0]*M
#average time from scheduled appointment start to patient exit, for each simulation run
avg_time_sch=[0]*M
#percentage of time exam room is being used for the exam, for each simulation run
perc_exam_time=[0]*M 
#percentage of time the exam room is used for preparing the patient
perc_ptprep_time=[0]*M
#percentatage of time the exam room is used for preparing the exam room
perc_roomprep_time=[0]*M

#patient scheduled arrival times (every 30 minutes), for n_max patients
PA_sch=list(range(0,(n_max)*30,30))

#patients to discard from calculations as warmup 
warmup = 100  

for m in range(0,M):
  EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E = run_radiology_sim(n_max=n_max)
  #to get average time, subtract all patient exam end times from patient arrival times, then average
  avg_time[m]=mean(array(EE[warmup:])-array(PA[warmup:]))
  #similarly, subtract all patient exam end times from patient scheduled arrival times, then average
  avg_time_sch[m]=mean(array(EE[warmup:])- array(PA_sch[warmup:]))
  #divide sum of all exam times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_exam_time=sum(E[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all patient prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_ptprep_time=sum(PP[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all exam prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_roomprep_time=sum(EP[warmup:])/(EE[-1]- PA[warmup])
  
  # if m is evenly divisible by 1000, print progress
  if m % 1000 == 0:
    print("run %s complete" % m)

##compute averages of metrics across all simulation runs
mean_avg_time_fastexam = mean(avg_time) #aggregate metric I
mean_avg_time_sch_fastexam  = mean(avg_time_sch) #aggregate metric II
mean_perc_exam_time_fastexam  =mean(perc_exam_time) #aggregate metric III
mean_perc_ptprep_time_fastexam  = mean(perc_ptprep_time) #aggregate metric IV
mean_perc_roomprep_time_fastexam  =  mean(perc_roomprep_time) #aggregate metric V

print("Average time in dept: %s minutes" % mean_avg_time_fastexam )
print("Average scheduled arrival to end exam: %s minutes" % mean_avg_time_sch_fastexam )
print("Average percent exam time: %s" % (mean_perc_exam_time_fastexam *100))
print("Average percent patient prep: %s" % (mean_perc_ptprep_time_fastexam*100))
print("Average percent room prep: %s" % (mean_perc_roomprep_time_fastexam *100))

run 0 complete
run 1000 complete
run 2000 complete
run 3000 complete
run 4000 complete
run 5000 complete
run 6000 complete
run 7000 complete
run 8000 complete
run 9000 complete
Average time in dept: 51.37598027397762 minutes
Average scheduled arrival to end exam: 51.375445088060445 minutes
Average percent exam time: 33.136841991329305
Average percent patient prep: 50.59767164804268
Average percent room prep: 16.668431585175707


Now, **fill in the code below, using the code after the first simulation as a template, to compute the difference in averages and the savings in seconds per dollar invested.** 

In [100]:
#difference in averages
print() 
#savings in seconds per dollar invested
print() 

Yet another possible recommendation is to add a new ‘patient preparation room’ so that the
patient preparation can start even before the exam room becomes available (i.e., even before the
previous exam is finished). Adding the new room will cost $20 per patient. How much will this
reduce the average time spent by the patients in the radiology department? How much is the saving
per dollar spent?

First, restore the original exam room prep time function by running the cell below. 

In [101]:
def get_exam_room_prep_time():
  return 10

Next, run the cell below, which removes the `max` statement, since we no longer need to consider the previous patient's exit time. 

In [102]:
def get_patient_prep_start_time(PA,FF,EE,n):
   return PA[n]+FF[n]

Now, rerun the simulations, now appending `_preproom` to the aggregate metric variable names. 

In [103]:
from numpy import mean, array

#define a random seed (so that we get the same outcome every time. )
random.seed(42)

#total number of simulations to run
M=10000
#number of patients per simulation
n_max = 1000

#average time per patient in the radiology department, for each simulation run
avg_time=[0]*M
#average time from scheduled appointment start to patient exit, for each simulation run
avg_time_sch=[0]*M
#percentage of time exam room is being used for the exam, for each simulation run
perc_exam_time=[0]*M 
#percentage of time the exam room is used for preparing the patient
perc_ptprep_time=[0]*M
#percentatage of time the exam room is used for preparing the exam room
perc_roomprep_time=[0]*M

#patient scheduled arrival times (every 30 minutes), for n_max patients
PA_sch=list(range(0,(n_max)*30,30))

#patients to discard from calculations as warmup 
warmup = 100  

for m in range(0,M):
  EE, ER, EP, PPS, PA, FF, PP, PR, PPS, ES, E = run_radiology_sim(n_max=n_max)
  #to get average time, subtract all patient exam end times from patient arrival times, then average
  avg_time[m]=mean(array(EE[warmup:])-array(PA[warmup:]))
  #similarly, subtract all patient exam end times from patient scheduled arrival times, then average
  avg_time_sch[m]=mean(array(EE[warmup:])- array(PA_sch[warmup:]))
  #divide sum of all exam times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_exam_time=sum(E[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all patient prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_ptprep_time=sum(PP[warmup:])/(EE[-1]-PA[warmup])
  #divide sum of all exam prep times by the total time (last exam end time minus first non-warmup patient arrival)
  perc_roomprep_time=sum(EP[warmup:])/(EE[-1]- PA[warmup])
  
  # if m is evenly divisible by 1000, print progress
  if m % 1000 == 0:
    print("run %s complete" % m)

##compute averages of metrics across all simulation runs
mean_avg_time_preproom = mean(avg_time) #aggregate metric I
mean_avg_time_sch_preproom  = mean(avg_time_sch) #aggregate metric II
mean_perc_exam_time_preproom  =mean(perc_exam_time) #aggregate metric III
mean_perc_ptprep_time_preproom  = mean(perc_ptprep_time) #aggregate metric IV
mean_perc_roomprep_time_preproom  =  mean(perc_roomprep_time) #aggregate metric V

print("Average time in dept: %s minutes" % mean_avg_time_preproom )
print("Average scheduled arrival to end exam: %s minutes" % mean_avg_time_sch_preproom )
print("Average percent exam time: %s" % (mean_perc_exam_time_preproom *100))
print("Average percent patient prep: %s" % (mean_perc_ptprep_time_preproom*100))
print("Average percent room prep: %s" % (mean_perc_roomprep_time_preproom*100))

run 0 complete
run 1000 complete
run 2000 complete
run 3000 complete
run 4000 complete
run 5000 complete
run 6000 complete
run 7000 complete
run 8000 complete
run 9000 complete
Average time in dept: 36.793799811569016 minutes
Average scheduled arrival to end exam: 36.79326462565186 minutes
Average percent exam time: 33.136841991329305
Average percent patient prep: 50.59767164804268
Average percent room prep: 33.336863170351414


Now, **fill in the code below, using the code after the first simulation as a template, to compute the difference in averages and the savings in seconds per dollar invested.** 

In [104]:
#difference in averages
print(mean_avg_time-mean_avg_time_preproom) 
#savings in seconds per dollar invested
print((mean_avg_time-mean_avg_time_preproom)*60/10) 

30.52611411489933
183.15668468939597
